In [1]:
import pandas as pd
import ssl
import matplotlib.pyplot as plt
import numpy as np
import hashlib
import boto3

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
pd.set_option('display.max_columns', 500)

In [4]:
ssl._create_default_https_context = ssl._create_unverified_context

# Tooling

### Upload S3

In [5]:
def UploadS3(filepath, key):
    s3 = boto3.client("s3")
    bucket_name = "economia-popular-delivery-content-indices"
    s3.upload_file(
        Filename=filepath,
        Bucket=bucket_name,
        Key=key,
        ExtraArgs={
            "ACL": "public-read",
            "ServerSideEncryption": "AES256"
        }
    )

    return True

### Hash String

In [6]:
def create_hash(text):
    return hashlib.sha256(text.encode()).hexdigest()

### IBGE parser

In [7]:
def ibge_dataframe_builder(df, items, classification, year):
    year = "{}".format(year)
    for item in items:
        dimensao = item["localidade"]["nivel"]["nome"]
        localidade = item["localidade"]["nome"]
        populacao = item["serie"]["{}".format(year)]
        id = create_hash(year + "-" + dimensao + "-" + localidade)
        
        row = {
            "id": id,
            "ano": year,
            "dimensao": dimensao,
            "localidade": localidade,
            "cor_raca": classification, 
            "populacao": populacao
        }
        df = df.append(row, ignore_index=True)        
    return df

## Dataframes

### Criando a estrutura base do Dataframe Vazia

In [8]:
columns = ["id", "ano", "dimensao", "localidade" , "cor_raca", "populacao"]
df = pd.DataFrame(columns=columns)
print(df)

Empty DataFrame
Columns: [id, ano, dimensao, localidade, cor_raca, populacao]
Index: []


In [9]:
columns_consolidado = ["id", "ano", "dimensao", "localidade" , "populacao_branca", "populacao_negra", "populacao_amarela", "populacao_parda","populacao_indigena" ,"populacao_sem_declaracao" ,"populacao_total"]
df_consolidado = pd.DataFrame(columns=columns_consolidado)
print(df_consolidado)

Empty DataFrame
Columns: [id, ano, dimensao, localidade, populacao_branca, populacao_negra, populacao_amarela, populacao_parda, populacao_indigena, populacao_sem_declaracao, populacao_total]
Index: []


In [10]:
# df = pd.read_json("https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N7%5Ball%5D%7CN3%5Ball%5D&classificacao=86%5Ball%5D%7C12056%5B99566%5D%7C58%5B0%5D")

In [11]:
df.head()

,id,ano,dimensao,localidade,cor_raca,populacao


In [12]:
df.tail()

,id,ano,dimensao,localidade,cor_raca,populacao


In [13]:
df.shape

(0, 6)

### URL's de Consulta no IBGE - Brasil

In [14]:
brasil_total_url = "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N1[all]&classificacao=86[0]|12056[99566]"
brasil_branca_url = "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N1[all]&classificacao=86[2776]|12056[99566]"
brasil_preta_url = "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N1[all]&classificacao=86[2777]|12056[99566]"
brasil_amarela_url = "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N1[all]&classificacao=86[2778]|12056[99566]"
brasil_parda_url = "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N1[all]&classificacao=86[2779]|12056[99566]"
brasil_indigena_url = "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N1[all]&classificacao=86[2780]|12056[99566]"
brasil_sem_declaracao_url = "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N1[all]&classificacao=86[2781]|12056[99566]"


### URL's de Consulta no IBGE - Região Metropolitanas

In [15]:
brasil_url = "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N1[all]&classificacao=86[all]|12056[99566]"
total_url = "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N7[all]|N3[all]&classificacao=86[0]|12056[99566]|58[0]"
branca_url = "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N7[all]|N3[all]&classificacao=86[2776]|12056[99566]|58[0]"
preta_url = "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N7[all]|N3[all]&classificacao=86[2777]|12056[99566]|58[0]"
amarela_url = "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N7[all]|N3[all]&classificacao=86[2778]|12056[99566]|58[0]" 
parda_url = "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N7[all]|N3[all]&classificacao=86[2779]|12056[99566]|58[0]"
indigena_url = "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N7[all]|N3[all]&classificacao=86[2780]|12056[99566]|58[0]"
sem_declaracao_url = "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N7[all]|N3[all]&classificacao=86[2781]|12056[99566]|58[0]"

### População Brasil - Total

In [16]:
df_temp = pd.read_json(brasil_total_url)

In [17]:
series = df_temp.resultados[0][0]['series']

In [18]:
df = ibge_dataframe_builder(df, series, "total", 2010)

### População Brasil - Branca

In [19]:
df_temp = pd.read_json(brasil_branca_url)

In [20]:
series = df_temp.resultados[0][0]['series']

In [21]:
df = ibge_dataframe_builder(df, series, "branca", 2010)

### População brasil - Preta

In [22]:
df_temp = pd.read_json(brasil_preta_url)

In [23]:
series = df_temp.resultados[0][0]['series']

In [24]:
df = ibge_dataframe_builder(df, series, "preta", 2010)

### População Brasil - Amarela

In [25]:
df_temp = pd.read_json(brasil_amarela_url)

In [26]:
series = df_temp.resultados[0][0]['series']

In [27]:
df = ibge_dataframe_builder(df, series, "amarela", 2010)

### População Brasil - Parda

In [28]:
df_temp = pd.read_json(brasil_parda_url)

In [29]:
series = df_temp.resultados[0][0]['series']

In [30]:
df = ibge_dataframe_builder(df, series, "parda", 2010)

### População Brasileira - Indigena

In [31]:
df_temp = pd.read_json(brasil_indigena_url)

In [32]:
series = df_temp.resultados[0][0]['series']

In [33]:
df = ibge_dataframe_builder(df, series, "indigena", 2010)

### População Brasileira - Sem Declaração

In [34]:
df_temp = pd.read_json(brasil_sem_declaracao_url)

In [35]:
series = df_temp.resultados[0][0]['series']

In [36]:
df = ibge_dataframe_builder(df, series, "sem_declaracao", 2010)

### População Total

In [37]:
df_temp = pd.read_json(total_url)

In [38]:
series = df_temp.resultados[0][0]['series']

In [39]:
df = ibge_dataframe_builder(df, series, "total", 2010)

In [40]:
df.shape

(70, 6)

### População Declarada como Branca

In [41]:
df_temp = pd.read_json(branca_url)

In [42]:
series = df_temp.resultados[0][0]['series']

In [43]:
df = ibge_dataframe_builder(df, series, "branca", 2010)

In [44]:
df.shape

(133, 6)

### População declarada como preta

In [45]:
df_temp = pd.read_json(preta_url)

In [46]:
series = df_temp.resultados[0][0]['series']

In [47]:
df = ibge_dataframe_builder(df, series, "preta", 2010)

In [48]:
df.shape

(196, 6)

### Populacão declarada como amarela

In [49]:
df_temp = pd.read_json(amarela_url)

In [50]:
series = df_temp.resultados[0][0]['series']

In [51]:
df = ibge_dataframe_builder(df, series, "amarela", 2010)

In [52]:
df.shape

(259, 6)

### População Parda

In [53]:
df_temp = pd.read_json(parda_url)

In [54]:
series = df_temp.resultados[0][0]['series']

In [55]:
df = ibge_dataframe_builder(df, series, "parda", 2010)

In [56]:
df.shape

(322, 6)

### População Indigena

In [57]:
df_temp = pd.read_json(indigena_url)

In [58]:
series = df_temp.resultados[0][0]['series']

In [59]:
df = ibge_dataframe_builder(df, series, "indigena", 2010)

In [60]:
df.shape

(385, 6)

### População Sem Declaração

In [61]:
df_temp = pd.read_json(sem_declaracao_url)

In [62]:
series = df_temp.resultados[0][0]['series']

In [63]:
df = ibge_dataframe_builder(df, series, "sem_declaracao", 2010)

In [64]:
df.shape

(448, 6)

## Sumário Exploratório

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 448 entries, 0 to 447
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          448 non-null    object
 1   ano         448 non-null    object
 2   dimensao    448 non-null    object
 3   localidade  448 non-null    object
 4   cor_raca    448 non-null    object
 5   populacao   448 non-null    object
dtypes: object(6)
memory usage: 21.1+ KB


In [66]:
df.head()

,id,ano,dimensao,localidade,cor_raca,populacao
0,47a213f3ebe30b5043086d3ead404042cf5c781d0d91e8...,2010,Brasil,Brasil,total,93504659
1,47a213f3ebe30b5043086d3ead404042cf5c781d0d91e8...,2010,Brasil,Brasil,branca,46018934
2,47a213f3ebe30b5043086d3ead404042cf5c781d0d91e8...,2010,Brasil,Brasil,preta,7764533
3,47a213f3ebe30b5043086d3ead404042cf5c781d0d91e8...,2010,Brasil,Brasil,amarela,1072443
4,47a213f3ebe30b5043086d3ead404042cf5c781d0d91e8...,2010,Brasil,Brasil,parda,38335331


In [67]:
df.tail()

,id,ano,dimensao,localidade,cor_raca,populacao
443,d23860d67f080e8454155455c2cc47c34f2637b24ba27f...,2010,Unidade da Federação,Rio Grande do Sul,sem_declaracao,34
444,29cc530b8162c0ff5f37a34a07dc7f2dc71e212ef73917...,2010,Unidade da Federação,Mato Grosso do Sul,sem_declaracao,-
445,f9a89474f5ba626244505fa48a4e9bbf243418ecfbe0ff...,2010,Unidade da Federação,Mato Grosso,sem_declaracao,-
446,3991c5a6b9fc6bf8aac55aa0cd131e551d2a6e4405ac8e...,2010,Unidade da Federação,Goiás,sem_declaracao,13
447,be5a3f4e95d5b5c829f8a46a27cafba1d52ef88bc46870...,2010,Unidade da Federação,Distrito Federal,sem_declaracao,-


In [68]:
df.cor_raca.value_counts()

total             64
branca            64
preta             64
amarela           64
parda             64
indigena          64
sem_declaracao    64
Name: cor_raca, dtype: int64

In [69]:
df.id.value_counts()

47a213f3ebe30b5043086d3ead404042cf5c781d0d91e81587c7a8284d1a64ae    7
5ce1397542c815696a781695e1064515dcc99a2f76dc135a4ce586f19bd1e2d5    7
6cb05b8aef5d15baf4c9d2f36b0f70b5bb2dee9c0f54078c134c112825270748    7
9a2eef2446115a88e99b7d170733f20c54f5de3ad5ec8799ca360ea2f0772ab0    7
f8102abf1ca27828da51f064f38f708ba345e30552cef85dc920fe3e0b8a4dcd    7
                                                                   ..
bdef4f57c6b9b71e8fdcfe81cdc11c79caaaad92970459c540ad9b0c96e259a5    7
1ee5cc191a2d21acde5f9faccbe4dde5116e6d3b19502b389ebaa072c41f27ee    7
5cd9f04b5a5280cabfa0ee7ee7765b73d6ce503bd69c4f182855286d336e74b6    7
29df00795eeae28e5ab5b69ee2205119d8e2fc77adff9812882b443014449803    7
be5a3f4e95d5b5c829f8a46a27cafba1d52ef88bc468702a63bb49b13a836fb6    7
Name: id, Length: 64, dtype: int64

# Consolidação dos Dados

## Criando o DF de referencia

In [70]:
df_consolidado.id = df[["id"]].copy()

In [71]:
df_consolidado.head()

,id,ano,dimensao,localidade,populacao_branca,populacao_negra,populacao_amarela,populacao_parda,populacao_indigena,populacao_sem_declaracao,populacao_total
0,47a213f3ebe30b5043086d3ead404042cf5c781d0d91e8...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,47a213f3ebe30b5043086d3ead404042cf5c781d0d91e8...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,47a213f3ebe30b5043086d3ead404042cf5c781d0d91e8...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,47a213f3ebe30b5043086d3ead404042cf5c781d0d91e8...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,47a213f3ebe30b5043086d3ead404042cf5c781d0d91e8...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
df_consolidado.id.value_counts()

47a213f3ebe30b5043086d3ead404042cf5c781d0d91e81587c7a8284d1a64ae    7
5ce1397542c815696a781695e1064515dcc99a2f76dc135a4ce586f19bd1e2d5    7
6cb05b8aef5d15baf4c9d2f36b0f70b5bb2dee9c0f54078c134c112825270748    7
9a2eef2446115a88e99b7d170733f20c54f5de3ad5ec8799ca360ea2f0772ab0    7
f8102abf1ca27828da51f064f38f708ba345e30552cef85dc920fe3e0b8a4dcd    7
                                                                   ..
bdef4f57c6b9b71e8fdcfe81cdc11c79caaaad92970459c540ad9b0c96e259a5    7
1ee5cc191a2d21acde5f9faccbe4dde5116e6d3b19502b389ebaa072c41f27ee    7
5cd9f04b5a5280cabfa0ee7ee7765b73d6ce503bd69c4f182855286d336e74b6    7
29df00795eeae28e5ab5b69ee2205119d8e2fc77adff9812882b443014449803    7
be5a3f4e95d5b5c829f8a46a27cafba1d52ef88bc468702a63bb49b13a836fb6    7
Name: id, Length: 64, dtype: int64

In [73]:
df_consolidado = df_consolidado.drop_duplicates(subset='id', keep='first')

In [74]:
df_consolidado.id.value_counts()

47a213f3ebe30b5043086d3ead404042cf5c781d0d91e81587c7a8284d1a64ae    1
5ce1397542c815696a781695e1064515dcc99a2f76dc135a4ce586f19bd1e2d5    1
6cb05b8aef5d15baf4c9d2f36b0f70b5bb2dee9c0f54078c134c112825270748    1
9a2eef2446115a88e99b7d170733f20c54f5de3ad5ec8799ca360ea2f0772ab0    1
f8102abf1ca27828da51f064f38f708ba345e30552cef85dc920fe3e0b8a4dcd    1
                                                                   ..
bdef4f57c6b9b71e8fdcfe81cdc11c79caaaad92970459c540ad9b0c96e259a5    1
1ee5cc191a2d21acde5f9faccbe4dde5116e6d3b19502b389ebaa072c41f27ee    1
5cd9f04b5a5280cabfa0ee7ee7765b73d6ce503bd69c4f182855286d336e74b6    1
29df00795eeae28e5ab5b69ee2205119d8e2fc77adff9812882b443014449803    1
be5a3f4e95d5b5c829f8a46a27cafba1d52ef88bc468702a63bb49b13a836fb6    1
Name: id, Length: 64, dtype: int64

In [75]:
df_consolidado.index = df_consolidado.id

In [76]:
df_consolidado = df_consolidado.drop(columns="id", axis=1)

In [77]:
df_consolidado.head()

,ano,dimensao,localidade,populacao_branca,populacao_negra,populacao_amarela,populacao_parda,populacao_indigena,populacao_sem_declaracao,populacao_total
id,,,,,,,,,,
47a213f3ebe30b5043086d3ead404042cf5c781d0d91e81587c7a8284d1a64ae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5ce1397542c815696a781695e1064515dcc99a2f76dc135a4ce586f19bd1e2d5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
225ca007c01f71a0bfdc0ca7dbe77c26e9424c109f157378bad9b2605f7867bb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
a680f96119c12ab0d4629f77563b6bb340d456ebb214990360faaebf9f9f00b2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cf0c0c0e84255199f73093a9352d5c209c32d71128ff395a2c26d163c0bfd235,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Inputando os dados quantitativos da população

In [78]:
for index, row in df.iterrows():
    # print(row.id)
    df_consolidado.loc[row.id, "ano"] = row.ano
    df_consolidado.loc[row.id, "dimensao"] = row.dimensao
    df_consolidado.loc[row.id, "localidade"] = row.localidade

    if row.cor_raca == "total":
        df_consolidado.loc[row.id, "populacao_total"] = row.populacao
    elif row.cor_raca == "branca":
        df_consolidado.loc[row.id, "populacao_branca"] = row.populacao
    elif row.cor_raca == "preta":
        df_consolidado.loc[row.id, "populacao_negra"] = row.populacao
    elif row.cor_raca == "amarela":
        df_consolidado.loc[row.id, "populacao_amarela"] = row.populacao    
    elif row.cor_raca == "parda":
        df_consolidado.loc[row.id, "populacao_parda"] = row.populacao 
    elif row.cor_raca == "indigena":
        df_consolidado.loc[row.id, "populacao_indigena"] = row.populacao
    elif row.cor_raca == "sem_declaracao":
        df_consolidado.loc[row.id, "populacao_sem_declaracao"] = row.populacao 
    # break

In [79]:
df_consolidado.replace("-", 0, inplace=True)

In [80]:
df_consolidado.head()

,ano,dimensao,localidade,populacao_branca,populacao_negra,populacao_amarela,populacao_parda,populacao_indigena,populacao_sem_declaracao,populacao_total
id,,,,,,,,,,
47a213f3ebe30b5043086d3ead404042cf5c781d0d91e81587c7a8284d1a64ae,2010,Brasil,Brasil,46018934,7764533,1072443,38335331,313142,276,93504659
5ce1397542c815696a781695e1064515dcc99a2f76dc135a4ce586f19bd1e2d5,2010,Região Metropolitana,Manaus - AM,248071,47404,11502,656272,3236,0,966485
225ca007c01f71a0bfdc0ca7dbe77c26e9424c109f157378bad9b2605f7867bb,2010,Região Metropolitana,Belém - PA,247878,84605,8441,647451,1460,0,989834
a680f96119c12ab0d4629f77563b6bb340d456ebb214990360faaebf9f9f00b2,2010,Região Metropolitana,Macapá - AP,59739,22504,2631,142260,381,0,227515
cf0c0c0e84255199f73093a9352d5c209c32d71128ff395a2c26d163c0bfd235,2010,Região Metropolitana,Grande São Luís - MA,177632,92413,7621,360578,1287,0,639530


### Convertendo as colunas para integer

In [81]:
df_consolidado["populacao_total"] = df_consolidado["populacao_total"].astype(int)
df_consolidado["populacao_branca"] = df_consolidado["populacao_branca"].astype(int)
df_consolidado["populacao_negra"] = df_consolidado["populacao_negra"].astype(int)
df_consolidado["populacao_amarela"] = df_consolidado["populacao_amarela"].astype(int)
df_consolidado["populacao_parda"] = df_consolidado["populacao_parda"].astype(int)
df_consolidado["populacao_indigena"] = df_consolidado["populacao_indigena"].astype(int)
df_consolidado["populacao_sem_declaracao"] = df_consolidado["populacao_sem_declaracao"].astype(int)

### Conferencia da Consolidação Quantitativa

In [82]:
df_consolidado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64 entries, 47a213f3ebe30b5043086d3ead404042cf5c781d0d91e81587c7a8284d1a64ae to be5a3f4e95d5b5c829f8a46a27cafba1d52ef88bc468702a63bb49b13a836fb6
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   ano                       64 non-null     object
 1   dimensao                  64 non-null     object
 2   localidade                64 non-null     object
 3   populacao_branca          64 non-null     int64 
 4   populacao_negra           64 non-null     int64 
 5   populacao_amarela         64 non-null     int64 
 6   populacao_parda           64 non-null     int64 
 7   populacao_indigena        64 non-null     int64 
 8   populacao_sem_declaracao  64 non-null     int64 
 9   populacao_total           64 non-null     int64 
dtypes: int64(7), object(3)
memory usage: 7.6+ KB


In [95]:
df_consolidado.iloc[0]

ano                                         2010
dimensao                                  Brasil
localidade                                Brasil
populacao_branca                        46018934
populacao_negra                          7764533
populacao_amarela                        1072443
populacao_parda                         38335331
populacao_indigena                        313142
populacao_sem_declaracao                     276
populacao_total                         93504659
populacao_branca_porcentagem               49.22
populacao_negra_porcentagem                  8.3
populacao_amarela_porcentagem               1.15
populacao_parda_porcentagem                 41.0
populacao_indigena_porcentagem              0.33
populacao_sem_declaracao_porcentagem         0.0
Name: 47a213f3ebe30b5043086d3ead404042cf5c781d0d91e81587c7a8284d1a64ae, dtype: object

In [96]:
df_consolidado.iloc[1]

ano                                                     2010
dimensao                                Região Metropolitana
localidade                                       Manaus - AM
populacao_branca                                      248071
populacao_negra                                        47404
populacao_amarela                                      11502
populacao_parda                                       656272
populacao_indigena                                      3236
populacao_sem_declaracao                                   0
populacao_total                                       966485
populacao_branca_porcentagem                           25.67
populacao_negra_porcentagem                              4.9
populacao_amarela_porcentagem                           1.19
populacao_parda_porcentagem                             67.9
populacao_indigena_porcentagem                          0.33
populacao_sem_declaracao_porcentagem                     0.0
Name: 5ce1397542c815696a

## Criando as porcentagens

In [84]:
df_consolidado['populacao_branca_porcentagem'] = ((df_consolidado['populacao_branca'] / df_consolidado['populacao_total']) * 100).round(2) 

In [85]:
df_consolidado['populacao_negra_porcentagem'] = ((df_consolidado['populacao_negra'] / df_consolidado['populacao_total']) * 100).round(2) 

In [86]:
df_consolidado['populacao_amarela_porcentagem'] = ((df_consolidado['populacao_amarela'] / df_consolidado['populacao_total']) * 100).round(2) 

In [87]:
df_consolidado['populacao_parda_porcentagem'] = ((df_consolidado['populacao_parda'] / df_consolidado['populacao_total']) * 100).round(2) 

In [88]:
df_consolidado['populacao_indigena_porcentagem'] = ((df_consolidado['populacao_indigena'] / df_consolidado['populacao_total']) * 100).round(2) 

In [89]:
df_consolidado['populacao_sem_declaracao_porcentagem'] = ((df_consolidado['populacao_sem_declaracao'] / df_consolidado['populacao_total']) * 100).round(2) 

In [90]:
df_consolidado.iloc[1]

ano                                                     2010
dimensao                                Região Metropolitana
localidade                                       Manaus - AM
populacao_branca                                      248071
populacao_negra                                        47404
populacao_amarela                                      11502
populacao_parda                                       656272
populacao_indigena                                      3236
populacao_sem_declaracao                                   0
populacao_total                                       966485
populacao_branca_porcentagem                           25.67
populacao_negra_porcentagem                              4.9
populacao_amarela_porcentagem                           1.19
populacao_parda_porcentagem                             67.9
populacao_indigena_porcentagem                          0.33
populacao_sem_declaracao_porcentagem                     0.0
Name: 5ce1397542c815696a

# Upload

## Salvando os Dados

In [91]:
df_consolidado.to_json('datasets/pessoas_economicamente_ativas.json', orient='records', lines=True)

In [92]:
df_consolidado.to_csv('datasets/pessoas_economicamente_ativas.csv', index=False)

In [93]:
UploadS3('datasets/pessoas_economicamente_ativas.json', 'sociais/pessoas_economicamente_ativas.pessoas_economicamente_ativas.json')

True

In [94]:
UploadS3('datasets/pessoas_economicamente_ativas.csv', 'sociais/pessoas_economicamente_ativas/pessoas_economicamente_ativas.csv')

True